# Using Python to analyze genome data

We will be making heavy use of the BioPython module. The BioPython module is a set of bioinformatics tools that attempt to make it as easy as possible to use Python for bioinformatics. The Bio module (along with some additional software) will enable us to collect sequences from online databases, parse them, align them, and analyze them. 

## What is a genome and why analyze it?
DNA is the chemical compound that contains the instructions needed to develop and direct the activites of organisms. Each DNA strand is made of chemical units called nucleotides that pair together to form a genetic alphabet. The combination of the A's, C's, T's, and G's which make up this alphabet determines the meaning of the information encoded by the strand of DNA. A genome is the complete set of all the DNA strands. Scientists are primarily concerned with those regions of the DNA which code for proteins. Proteins make up body structures like organs and tissue, as well as control chemical reactions and carry signals between cells. When the DNA of a cell becomes damaged or mutated, a defective protein may be produced. These defective proteins can disrupt normal processes in the body and can lead to disease. By studying the genetic code we can study the mutations that give rise to abnormal proteins. 

Genomics is not limited to the study of disease. DNA can be used to draw comparisons between individuals or entire species. The fields of comparitive and evolutionary genomics often use DNA or protein sequences to draw evolutionary relationships between organisms of different taxa in order to study biological similarites and differences between organisms.

## How do we study genomes?
Genomes are first sequenced to determine the exact order of their base pairs. The sequences are then assembled like a jigsaw puzzle in order to determine their linear order. Once these sequences are assembled we can start collecting information about particular genes. Often, the goal is to gather many homologous sequences (sequences that are shared between individuals - derived from a common ancestor) and compare them by looking for substitutions, deletions, or additional base pairs. These differences between sequences can reveal variations in functionality of proteins or ellucidate evolutionary relationships between taxa. The necessary steps for these types of analyses are usually as follows: 

1. Determine a sequence of interest (this could be a specific gene responsible for some type of disease of illness)
2. find homologous (similar/related) sequences by perfoming a BLAST search of a database containing sequence data and download them as FASTA files
3. align these sequences (called a multiple sequence alignment or MSA)
4. create a phylogenetic tree from the MSA
5. make estimates and measurements based off of this tree
6. other interesting things...


## Step 1: Search and download sequences from NCBI database

### import all of the necessary libraries
We will need:
* [BioPython](http://biopython.org/)
* urllib
* Subprocess
* Sys 

### Addtional programs
detailed below are some additional programs to be used in the analysis of genomic data. All of the programs are open source and freely available. For the purpose of this project we will assume the programs have been downloaded and when necessary, appended to your system PATH.

In [9]:
from Bio import Entrez
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio.Align.Applications import MuscleCommandline
from Bio import AlignIO
from Bio import Phylo

from urllib.error import HTTPError
import subprocess

### Set up our search query
The most important step in the analysis is determining what gene or protein we want to study. For the sake of this project let's look for a gene everyone knows, BRCA1. The BRCA1 gene encodes a nuclear phosphoprotein that plays a role in maintaining genomic stability and acts as a tumor surpressor. Mutations in this gene account for approximately 40% of inherited breast cancers and more than 80% of inherited breast and ovarian cancers [ReSeq May 2009]. More information about the BRCA1 & BRCA2 genes can be found here: https://www.cancer.gov/about-cancer/causes-prevention/genetics/brca-fact-sheet

### Set search terms and download raw sequences
We can use the .esearch method to specify our search query of the Entrez gene database and then download all of the sequences we find into .fasta files for later analysis.

In [3]:
search_term = 'BRCA1'
Entrez.email = 'tuc12093@temple.edu'
search_handle = Entrez.esearch(db="nucleotide",
                               term=search_term,
                               usehistory="y",
                               idtype="acc")
search_results = Entrez.read(search_handle)
search_handle.close()

acc_list = search_results["IdList"]
count = 10

webenv = search_results["WebEnv"]
query_key = search_results["QueryKey"]

batch_size = 3
outfile = search_term + ".fasta"
out_handle = open(outfile, "w")
for start in range(0, count, batch_size):
    end = min(count, start + batch_size)
    print("Going to download {} to {}".format(start+1, end))
    attempt = 0
    while attempt < 3:
        attempt += 1
        try:
            fetch_handle = Entrez.efetch(db="nucleotide",
                                         rettype="fasta", retmode="text",
                                         retstart=start, retmax=batch_size,
                                         webenv=webenv, query_key=query_key,
                                         idtype="acc")
        except HTTPError as err:
            if 500 <= err.code <= 599:
                print("Received error from server {}".format(err))
                print("Attempt {} of 3".format(attempt))
                time.sleep(15)
            else:
                raise
                
    data = fetch_handle.read()
    fetch_handle.close()
    out_handle.write(data)
    
out_handle.close()

Going to download 1 to 3
Going to download 4 to 6
Going to download 7 to 9
Going to download 10 to 10


### What are fasta files? How do we use them?
fasta files are like text files that contain nucleotide, protein or codon sequences. Though, fasta files have a specific format which distinguishes them from plain text files. Each sequence record in a fasta file begins with a single line description followed by lines of the sequence data. Each sequence record line must begin with '>'. Any other type of identifying information can follow in the sequence record line. 

example fasta file:

`>sequence 1 | sequence type | organism | other information
ACGTGCATCGCATTATTAGCATAGCAGACGATATAGCAGCATCATTAAAACAGACAGACATTTAGACAGACAGAGACGCGCGACGACTAAGCAGAAAAAGCAATTTTTTACG`

`>sequence 2 | sequence type | organism | other information
AGCATTAGACGCAGCAGCAGCGCGCATATTATTACAGACATTAGACATATTTAGATCAGATACATAGCACAGACAGAA`




### Parse the results file to get only the Homo sapiens result
We are really only interested in the homologs of the Homo sapiens BRCA1 gene and therefore, from our results, we want to create one file that contains only the fasta sequence of interest that we will then use in our BLAST search. BioPython makes parsing and extracting this sequence trivial.


The Seq.Record class in BioPython provides us with a way or extracting and parsing all of the data downloaded from sequences files. This class forms the basic data type for Bio.SeqIO. This class contains these useful objects (taken from the BioPython [documentation](http://biopython.org/DIST/docs/tutorial/Tutorial.pdf)). It will provide the basic structure for all of our sequence manipulation.

* .seq – The sequence itself, typically a Seq object.
* .id – The primary ID used to identify the sequence – a string. In most cases this is something like an accession number.
* .name – A “common” name/id for the sequence – a string. In some cases this will be the same as the accession number, but it could also be a clone name. I think of this as being analogous to the LOCUS id in a GenBank record.
* .description – A human readable description or expressive name for the sequence – a string.
* .letter annotations – Holds per-letter-annotations using a (restricted) dictionary of additional information about the letters in the sequence. The keys are the name of the information, and the information is contained in the value as a Python sequence (i.e. a list, tuple or string) with the same length as the sequence itself. This is often used for quality scores (e.g. Section 20.1.6) or secondary structure information (e.g. from Stockholm/PFAM alignment files).
* .annotations – A dictionary of additional information about the sequence. The keys are the name of the information, and the information is contained in the value. This allows the addition of more “unstructured” information to the sequence.
* .features – A list of SeqFeature objects with more structured information about the features on a sequence (e.g. position of genes on a genome, or domains on a protein sequence). The structure of sequence features is described below in Section 4.3. .dbxrefs - A list of database cross-references as strings.

In [4]:
for record in SeqIO.parse("BRCA1.fasta", "fasta"):
    if "Homo sapiens" in record.description:
        brca_hs = record
        break

with open("BRCA1_homo_sapiens.fasta", "w") as outfile:
    SeqIO.write(brca_hs, outfile, "fasta")

Our resulting file only contains the first record in the search that was from a human being.

if we want to check this we can use the code:

In [5]:
record = SeqIO.read("BRCA1_homo_sapiens.fasta", "fasta")
print(record)

ID: NM_030571.3
Name: NM_030571.3
Description: NM_030571.3 Homo sapiens Nedd4 family interacting protein 1 (NDFIP1), mRNA
Number of features: 0
Seq('GTAGTCCCCGCCTCTTCCCCAGGGGCCGCGTCGGAGCCTCGGCGGCGGCGGCGG...AAA', SingleLetterAlphabet())


## Step 2. BLAST the sequences
BLAST or Basic Local Alignment Search Tool attempts to find regions of local similarity between sequences. The program compares nucleotide or protein sequences to sequence databases and then calculates the statistical significance of the matches. The results from BLAST can then be used to infer evolutionary relationships as well as to determine members of a gene family.

The BLAST algorithm itself is complex. More information can be found on the [BLAST wikipedia page](https://en.wikipedia.org/wiki/BLAST). For our purposes, BLAST searches the entire NCBI database to find sequences that are very similar to our search sequence. We then assume that since these sequences are so similar (measured in % identity) that they are probably derived from the same common ancestor. With this knowledge we can then perform alignments of the sequences and other analyses (more on this below).

In [6]:
query = SeqIO.read("BRCA1_homo_sapiens.fasta", "fasta")
blast_results = NCBIWWW.qblast("blastn", "nt", query.seq)

with open('blast_results.xml', 'w') as f:
    f.write(blast_results.read())

This will write an XML file to the "blast_results.xml" file in our working directory. We can then parse this xml file for the records and write these records to a file for alignment.

In [7]:
result_handle = open("blast_results.xml")
blast_records = NCBIXML.parse(result_handle)

with open("blast_results.fasta", "w") as f:
    for i,blast_record in enumerate(blast_records):
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                f.write('>{}\n{}\n'.format(alignment.title, hsp.query))

## Step 3. Perform multiple sequence alignment (MSA) on the fasta files
Now that we have found homologous genes we can perform a multiple sequence alignment. But first, what is a MSA?

### MSA
MSAs are sequence alignments of three or more sequences or nucleotides or amino acids. In most cases, the sequences used as inputs to the MSA are assumed to be related (homologous). From the resultant MSA we can infer homology and perform phylogenetic analysis. In essence we are taking all of the input sequences and finding out which parts of them are all the same and which individual parts of a particular sequence might differ from the rest of the group.

ex. using amino acids
* seq 1 = M Q P I L L P
* seq 2 = M L R L P
* seq 3 = M P V I L K P

* aligned ==> M Q P I L L P
* aligned ==> M L R - L - P
* aligned ==> M P V I L K P

MSA programs perform these alignments and then derive scores for each based on the relatedness of the pairing. The MSA with the best score is then used as the output MSA. Algorithms and scoring matrices for MSAs can get complicated so we'll just ignore the details for now. Just know that we need to align the raw sequences for further analysis and that programs exist to do just that.

### using MUSCLE for multiple sequence alignment
#### NOTE: the following is assuming that this analysis is being run on a linux system
[MUSCLE](https://www.drive5.com/muscle/downloads.htm) is just one of many programs that can perform MSA. It exists as its own executable. Therefore, it is typically downloaded and then run from the command line.

After adding muscle to your path, it is run like so...

`muscle -in raw_fasta_files.fa -out aligned_fasta_files.afa`

But we can use Python to call muscle and perform the alignment within our script

`records = (r for r in SeqIO.parse("blast_results.fasta", "fasta"))`

`command = MuscleCommandline(input='BRCA1.fasta', out="aligned.afa")`

`child = subprocess.Popen(str(command),
                             stdout=subprocess.PIPE,
                             stderr=subprocess.PIPE,
                             universal_newlines=True,
                             shell=(sys.platform!="win32"))`
                             
`SeqIO.write(records, child.stdin, "fasta")`

`child.stdin.close()`

This will produce an aligned output file in fasta format. Again, we can use Bio to parse the alignment if we like.

In [8]:
align = AlignIO.read("aligned.afa", "fasta")
print(align)

SingleLetterAlphabet() alignment with 67 rows and 3670 columns
--------------------------------------------...--- gi|163644541|gb|AC206719.3|
--------------------------------------------...--- gi|3165402|gb|AC004752.1|
--------------------------------------------...--- gi|23396212|gb|AC011401.9|
--------------------------------------------...--- gi|163644541|gb|AC206719.3|
--------------------------------------------...--- gi|23396212|gb|AC011401.9|
--------------------------------------------...--- gi|163644541|gb|AC206719.3|
--------------------------------------------...--- gi|23396212|gb|AC011401.9|
--------------------------------------------...--- gi|163644541|gb|AC206719.3|
--------------------------------------------...--- gi|23396212|gb|AC011401.9|
--------------------------------------------...--- gi|23396212|gb|AC011401.9|
--------------------------------------------...--- gi|163644541|gb|AC206719.3|
--------------------------------------------...A-- gi|3165402|gb|AC004752.1

## Step 4. Create a phylogenetic tree from the alignment
After the MSA is completed and the file has been created for the alignment, a phylogenetic tree is the next step in the analysis pipeline.  A phylogenetic tree is a visual to express the relationship of species or in this case gene family.  The MSA is used along with a substitution model to determine the relationship of the genes or sequences of interest.  There are a number of software packages that have been developed to complete this step in the pipeline, some of which also carry out model selection to determine the best substitution model for the specific set of data.

BioPython cannot construct trees on its own so we will again have to call an outside program in order to construct a phylogenetic tree from our alignment file. Then, once the construction of the tree is complete, we can use BioPython to view the result.

#### Using RAxML to construct maximum likelihood trees
[RAxML](https://sco.h-its.org/exelixis/web/software/raxml/index.html) is just one of many programs for creating maximum likelihood trees. Maximum likelihood trees are trees that are constructed by selecting the tree that best maximizes the probability of observing the genetic data given a particular tree.

`command = "raxml -s aligned.afa -n brca1_ml -m GTRCAT -f a -x \$RANDOM -N autoMRE -p \$RANDOM" `

`subprocess.Popen(command)`

Again, for the purpose of this program it is assumed that you have installed RAxML and it is in your path as 'raxml'. We can use subprocess again to call RAxML by specifying the command as a string. In this command we are calling raxml to create ml results based on the 'aligned.afa' file. The results will all be files appended with 'brca1_ml'. -m specifies the substitution model. The '-f a' options will run a rapid bootstrap analysis and search for the maximum likelihood tree in one command. -x $RANDOM selects a random number to initiate a heuristic search. -N tells the program to automatically select the number of bootstrap replicates with autoMRE. -p generates another random number used in parsimony inference. 

We can now draw the tree we created using Bio.Phylo. the tree that we have created is in Newick format. Newick files esseintially are sets of nested parentheses describing the distances between branches and nodes of our created tree. There are much prettier ways to draw trees but the simplest is to just draw the tree using ascii characters.

In [5]:
tree = Phylo.read('brca1_ml_tree.nwk', 'newick')
Phylo.draw_ascii(tree)

  ________________ XM_006873258.1
 |
 |                     ______________ XM_004381130.2
 |                  __|
 |                 |  |__________________________ XM_010591427.1
 |                ,|
_|                || _____________________ XM_006891121.1
 |                |||
 |                | |__________________ XM_007939063.1
 |                |
 |                |  _________________________ XM_004452614.2
 |                | |
 |                | |               ____ XM_005324230.1
 |________________| |  ____________|
                  | | |            |_ XM_015498946.1
                  | | |
                  | | |    __________________ XM_008591647.1
                  | | |  _|
                  | | | | |____________________________ XM_004854313.1
                  | | | |
                  | | | | __________________________ XM_013020402.1
                  | | | ||
                  |_| | ||  _______________________ XM_014586401.1
                    | | ||_|
              

### Gene-Tree Species-Tree reconciliation - an example of possible analysis
After the construction of a phylogenetic tree, there are different steps that can be taken to look for specific evolutionary information.  One step is to perform gene tree and species tree reconciliation to root the gene tree created for a better understanding of relationship between the genes.  Software packages exist to perform this step.  To perform this a gene tree needs to be provided to the program which was created in the last step of analysis.  Also a species tree must be provided which can be obtained from a number of sources.  One reliable source is from the NCBI Taxonomy database in which the species of the genes of interest can be entered, upon entry a species tree is created, and can be downloaded into a format that is needed most likely FASTA format. 

One of the more popular programs for this kind of analysis is [Notung](https://www.cs.cmu.edu/~durand/Notung/). Notung can carry out its four main tasks: reconcile, rearrange, root and resolve, from the command line. In each case, Notung reads in gene and species trees (the input trees) and executes the specified task, resulting in one or more modified trees (the output tree(s)). This modified tree is written to a file. 

a typical command using notung takes the form:

`java -jar Notung-2.9.jar [input tree(s)] [task] [options]`

## Step 5. More analysis and file manipulation
Bioinformatics pipelines are useful techniques when it comes to analyzing and comparing a multitude of gene or protein sequences to one another. Analysis of structural conservation, sequence conservation, gene trees, and dN/dS values (which is an indicator of what type of selection is acting on a gene or protein) are useful in determining evolutionary relationships and similarities or differences amongst a set of sequence data. There is an abundance of software packages and servers available for carrying out a pipeline; however, formatting discrepancies, lack of computational power, software bugs, and a general outdated-ness of the available programs proves to be a crippling inconvenience. Our project sets out to simplify the process that each of the current programs available complicates. Our aim is to create a python program that gathers a set of genetic homologs via NCBI, cleans up these sequences and formats them correctly, builds reconciled and rooted gene trees, selects for an appropriate evolutionary model, and analyzes dN/dS selection rates acting on each gene in a given data set. 

Up to this point in the pipeline, the data that has been gathered includes:
* DNA sequences
* Amino Acid sequences 
* Multiple Sequence Alignments
* Phylogenetic trees


Furthermore, these data files should be saved either as a fasta file or as a phylip file. These formats have been manipulated in the beginning of the pipeline by Brian and Genaro. Moreover, these file types are output files from the software packages that are already made. This means that the output files from Multiple Sequence Alignment software can be piped from the “downloads” folder to a desired folder, already in the correct format. However, if for some reason the file format had not been changed by this point in the pipeline, we have written some code to read in the file and change its format.


The next step of the pipeline determines the file type to be input into the software. Some software packages require the input to be in fasta format, some require the file format to be in phylip format. It can be very frustrating when the software packages require particular file formats that you may not have, therefore we have written code that allows for agile file manipulation at almost each step of the pipeline. We have not exactly hard-coded any “file example” of what either fasta or a phylip format can be modeled after, so that might be a next step. However, we have created multiple check-points in our code to manipulate the file format to whichever desired type is needed for the next step. 


### File manipulation
The first step we have to address is formatting the files (that Gennaro and Brian have gathered) correctly. This involves accessing a terminal and a specific location in the terminal where the sequence files are located. Next, we have to open the files in question and make sure the proper mode ( read and write )is accounted for. Many of the existing software requires phylip formatting, so the next line of code changes the .txt extension to .rphylipi. This extension changes the file to a phylip interleaved file.

Once the file of sequences has been formatted properly, we can call on a program like HyPhy to perform data visualization. Hyphy can be installed in the terminal via pip install as shown in the code. Hyphy can be utilized to build phylogenetic trees, an important step in the pipeline, that can help when it comes to inferring how closely related genes are. Additionally, calling on certain tests in HyPhy in our python code will allow us to run several different analyses that will reveal dN/dS, selection trends, and evolutionary models that are relevant to the gene sequences in the original file. A GUI can be created and called upon for ease of access when operating HyPhy; however, we feel as though this is currently out of our skill set. HyPhy can also be accessed via DataMonkey.org. The following web page covers the installation procedure and GUI installation via the terminal:  https://github.com/veg/hyphy.

`import os`

`fn = 'GeneSeq.txt'`

`path = os.getcwd()`

`fpath = os.path.join(path, fn)`

`fh = open(fn, 'r+')`

`# read and write access aloows us to change the file format`

`newname = fh.replace('.txt', '.rphylip')`

`# this command was pulled from stackoverflow`

`pip install hyphy-python==0.1.5`

The output of this pipeline will be slightly different each time it is run depending on the desired input. For example, if an individual wants to test a hypothesis to examine one particular species the pipeline would be more tailored to handle DNA sequences and would utilize software packages unique to the analysis of DNA sequences. Likewise, if the hypothesis was to test protein divergence then the pipeline would be tailored to software inputs and outputs unique to the analysis of Amino Acid sequences. The implications of each analysis lead the ultimate output of the pipeline to vary. 

### DNA analysis using HyPhy
Using the example of the DNA sequence analysis, the pipeline might utilize a software package such as DATAMONKEY and HyPhy. Once the DNA sequences were collected, run through a multiple sequence alignment software, and saved in fasta format, the datafile can be input into HyPhy. At this point, the software package will return:

* A gene tree from the data
* dN/ds values
* selection of the gene within the lineage. 

This output file is often one large PDF sent to your downloads folder by the software. This output file is often sufficient for further analysis, however this is an area in our pipeline that we could further manipulate. We could write code to take the data output file and highlight or extract desired information. This part would be difficult because each hypothesis utilizes our pipeline differently, therefore the desired data from one pipeline might not be the same desired data from a different hypothesis. For example, one pipeline might want all the dN/ds values to be made visible, but another pipeline might be only interested in every site that has substituted one nucleotide to another. 

### Amino Acid Analysis with PAML
Using an example of the Amino Acid sequence analysis, the pipeline might utilize a software package such as PAML. The previous steps leading up to this point are very similar to the DNA sequence steps, gather sequences, align them, and save them in the desired format. For PAML to run its analysis, the phylip format is required for the multiple sequence alignment to be input into the PAML software. Under this analysis, the software package might return:

* A diverged gene tree
* Chi square values
* Selection on the branch 

This output file will be very similar to what the DNA sequence software will output. Again, you will run into the same problem by trying to manipulate the results in one desired way. This problem arises from the issue with each hypothesis being interested in different pieces of the output data. 

## Project Overview, Pitfalls, and Future Directions
The steps of the pipeline are as follows: 
1. Collect desired sequences (DNA or Amino Acid)
2. Write a python script to collect the sequences from databases
   1. NCBI 
   2. KEGG
   3. UniProt
3. Use the sequence files to pipe into a software package that will align your sequences. 
4. Collect the output file from this software and manipulate the file format such that the file can be piped into the next step in the analysis. 
5. Pipe the correct file formatted datafile into a software package that will further analyze data. This section will output a file that can be visualized, and that contains data that can be interpreted for results.

### future directions
The future directions of this project absolutely include version control, and agile development. At this point in time there is a wide range of hypotheses one could ask with the data, however, that means that writing one pipeline will not be sufficient to answer each question. That being said, it would be helpful to write a pipeline specific to only DNA based questions, and then Amino Acid based questions. Both of these areas can be further subdivided into many different questions. For that reason, we could create versions of our pipeline that best answer unique questions, instead of trying to use one version of the pipeline for many different questions. Another area we want to improve is coding the GUI for HyPhy. This version of our pipeline works well because it is less code-intensive which is nice for scientists that do not know a lot of code. Our next version of this code will have a more indepth, coded GUI. The total number of man hours needed to complete this are about 10-15 hours. Thus far, we have completed steps as far as writing python code to open the files and format them correctly. Additionally, we have written code to install the software package to carry out the future analyses. At this point in time, we do not need further external resources, as the BioPython website has been sufficient. For larger analyses, it might make more sense to use a supercomputer rather than a laptop.  

## References
Anisimova, Maria, et al. "State-of the art methodologies dictate new standards for phylogenetic analysis." (2013): 161.


Sergei L. Kosakovsky Pond, Simon D. W. Frost and Spencer V. Muse (2005) HyPhy: hypothesis testing using phylogenies Bioinformatics 21(5): 676-679
Veg. “Veg/Hyphy.” GitHub, 17 Nov. 2017, github.com/veg/hyphy.